# **Receipt OCR solutions for document automation**
  
# 문서 자동화를 위한 영수증 OCR 솔루션

## 1. Installation

In [1]:
!pip install easyocr


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install ultralytics  
!pip install opencv-python-headless  
from ultralytics import YOLO

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/d0/a3/87a9d3c9d1e873d922409d18faa2a77c18e3f7a32757fdaca16bb7a889f3/ultralytics-8.3.49-py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.23.0 from https://files.pythonhosted.org/packages/75/5b/ca6c8bd14007e5ca171c7c03102d17b4f4e0ceb53957e8c44343a9546dcc/numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Obtaining dependency information for opencv-python>=4.6.0 from https://files.pythonhosted.org/packages/66/82/564168a349148298aca281e342551404ef5521f33fba17b388ead0a84dc5/opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata
  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Obtaining dependency information for tqdm>=4.64.0 from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqd

In [1]:
!pip install paddlepaddle
!pip install paddleocr


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## 2. YOLO Text Detection Training
   - dataset : 400 images of English and Korean receipts
   - Using colab GPU

In [ ]:
model = YOLO('yolov8m.pt')  # YOLOv8m 사용

model.train(
    data="/content/drive/MyDrive/AISW_project/dataset/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name="receipt_detection",
    device=0,
    patience = 20,
    augment = True
)

## 2.2. YOLO Text Detection Visualization

In [2]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np

# YOLOv8 모델 불러오기 (best.pt 파일 경로 지정)
model = YOLO('best.pt')  # 학습된 best.pt 경로

# 테스트 이미지 경로
test_image_path = 'test_image.jpg'

# 이미지 로드
img = cv2.imread(test_image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 모델 예측
results = model(img_rgb)

# 첫 번째 결과 객체 가져오기 (list 형식이므로 첫 번째 결과에 접근)
result = results[0]

# 예측 결과 시각화
result.show()  # 예측된 결과를 이미지에 시각화

# 예측된 바운딩 박스 출력 (클래스, confidence, 바운딩 박스 좌표)
boxes = result.boxes  # 예측된 객체들
for box in boxes:
    # 바운딩 박스를 넘파이 배열로 변환
    xywh = box.xywh.cpu().numpy()  # `.cpu().numpy()`로 텐서를 넘파이 배열로 변환
    cls = box.cls.item()  # 클래스 ID
    confidence = box.conf.item()  # 신뢰도

    print(f"Class: {cls}, Confidence: {confidence:.2f}, Bounding Box (xywh): {xywh}")



0: 640x480 1 date_time, 3 items, 1 receipt, 1 store, 1 total, 851.8ms
Speed: 11.9ms preprocess, 851.8ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 480)
Class: 2.0, Confidence: 0.99, Bounding Box (xywh): [[     1139.5      1511.1        1430      3022.2]]
Class: 0.0, Confidence: 0.89, Bounding Box (xywh): [[     956.09      894.98      542.34      92.471]]
Class: 3.0, Confidence: 0.87, Bounding Box (xywh): [[     672.51      484.58       433.9      127.55]]
Class: 1.0, Confidence: 0.84, Bounding Box (xywh): [[     660.42      1141.2      369.19      83.394]]
Class: 4.0, Confidence: 0.81, Bounding Box (xywh): [[     1089.3      1594.2      1234.8      83.819]]
Class: 1.0, Confidence: 0.77, Bounding Box (xywh): [[     593.73      1212.5      248.27      69.255]]
Class: 1.0, Confidence: 0.27, Bounding Box (xywh): [[     753.65      1282.1      569.57      82.491]]


## 2.2. YOLO Text Detection Testing  
- dataset : 30 images of Korean receipts
- Using colab GPU

In [ ]:
from ultralytics import YOLO
import os
from pathlib import Path
import shutil
import yaml
import cv2
from matplotlib import pyplot as plt
import numpy as np

# 테스트 데이터셋 경로 설정
TEST_IMAGES_DIR = "/content/drive/MyDrive/AISW_project/test/test/images"  # 테스트 이미지가 저장된 디렉토리 경로
TEST_LABELS_DIR = "/content/drive/MyDrive/AISW_project/test/test/labels"  # 테스트 라벨 파일이 저장된 디렉토리 경로 (YOLO 포맷)
MODEL_PATH = "/content/best.pt"  # 학습된 모델 경로

# YOLOv8 모델 로드
model = YOLO(MODEL_PATH)

# 테스트 데이터셋 준비
if not os.path.exists(TEST_IMAGES_DIR):
    raise FileNotFoundError(f"Test images directory not found: {TEST_IMAGES_DIR}")
if not os.path.exists(TEST_LABELS_DIR):
    raise FileNotFoundError(f"Test labels directory not found: {TEST_LABELS_DIR}")

# YOLOv8에서 평가를 위해 데이터셋 설정 파일 생성
DATA_CONFIG_PATH = "dataset.yaml"
data_config = {
    'path': '.',
    'train': '',  # 평가 단계에서는 필요 없음
    'val': '',    # 평가 단계에서는 필요 없음
    'test': TEST_IMAGES_DIR,
    'names': {0: 'class_0', 1: 'class_1', 2: 'class_2', 3: 'class_3', 4: 'class_4'}  # 클래스 이름 설정
}

with open(DATA_CONFIG_PATH, 'w') as f:
    yaml.dump(data_config, f)

# 모델 평가
results = model.val(data=DATA_CONFIG_PATH, imgsz=640, split='test')

# 모델 평가 결과 출력
print("\nEvaluation Results:")
print(f"Precision (mAP@0.5): {results.box.map50:.4f}")
print(f"Precision (mAP@0.5:0.95): {results.box.map:.4f}")
print(f"Overall Fitness: {results.box.fitness():.4f}")
print(f"Recall (Mean): {results.box.mr():.4f}")  # Mean Recall


# 시각화
print("\nGenerating Detection Results...")
PREDICTIONS_DIR = "runs/predict/test"
shutil.rmtree(PREDICTIONS_DIR, ignore_errors=True)
model.predict(source=TEST_IMAGES_DIR, save=True, save_txt=True, conf=0.25)

# 결과 이미지 시각화
def visualize_results(image_dir, results_dir):
    image_paths = list(Path(image_dir).rglob("*.jpg"))  # jpg 이미지만 사용
    result_images = list(Path(results_dir).rglob("*.jpg"))

    for image_path, result_image_path in zip(image_paths, result_images):
        # 원본 이미지 로드
        original_image = cv2.imread(str(image_path))
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

        # 결과 이미지 로드
        result_image = cv2.imread(str(result_image_path))
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)

        # 시각화
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.title("Original Image")
        plt.imshow(original_image)
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.title("Detection Result")
        plt.imshow(result_image)
        plt.axis("off")

        plt.show()

visualize_results(TEST_IMAGES_DIR, PREDICTIONS_DIR)

## 3. Pre-Processing

In [5]:
def preprocess_for_ocr(image):
    """
    이미지를 전처리하여 OCR 성능을 향상
    그레이스케일, 이진화, 노이즈 제거, 대비 조정 등을 포함
    """
    # 1. 그레이스케일 변환
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 2. 이진화 (Otsu의 이진화 기법 사용)
    _, binarized_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 3. 노이즈 제거 (GaussianBlur 사용)
    denoised_image = cv2.GaussianBlur(binarized_image, (5, 5), 0)

    # 4. 대비 조정 (이미지 대비 향상)
    contrast_image = cv2.convertScaleAbs(denoised_image, alpha=1.5, beta=0)

    # 5. 기울기 보정 (이미지 회전 보정)
    coords = np.column_stack(np.where(contrast_image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = contrast_image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(contrast_image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated_image

# 전처리를 거친 이미지를 EasyOCR로 텍스트 인식
def ocr_with_preprocessing(image, reader):
    
    # 전처리 함수 호출
    preprocessed_image = preprocess_for_ocr(image)

    # EasyOCR로 텍스트 인식
    ocr_result = reader.readtext(preprocessed_image)

    return ocr_result


In [3]:
# 이미지 크기 조정 함수 (이미지 확대하여 OCR 성능을 향상)
def resize_for_ocr(image):
    height, width = image.shape[:2]
    new_width = width * 2  # 두 배로 확대
    new_height = height * 2
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    return resized_image

## 4. YOLO_EasyOCR Pipeline  
- store, date_time, total : 1 Bounding Box with Highest Confidence 
- Item : All Bounding Box with Confidence > 0.5

In [9]:
import cv2
from ultralytics import YOLO
from easyocr import Reader
from math import atan2, degrees
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# YOLOv8 모델 로드
model = YOLO('best.pt') 

# 테스트 이미지 경로
image_path = "test_image.jpg"
image = cv2.imread(image_path)

# YOLOv8으로 이미지 처리
results = model(image)

# 클래스별로 검출 결과 저장
detections_by_class = {0: [], 1: [], 3: [], 4: []}

# 임계값 설정 (신뢰도 기준)
confidence_threshold = 0.45

# YOLO 모델 결과에서 바운딩 박스를 추출
for result in results:
    boxes = result.boxes  # YOLOv8 결과에서 박스를 추출
    for box in boxes:
        confidence = box.conf[0].item()
        class_id = int(box.cls[0].item())

        #
        if confidence >= confidence_threshold:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # 좌표 추출
            #클래스 0,1,3,4에 대해서만 처리
            if class_id in detections_by_class:
                detections_by_class[class_id].append((confidence, x1, y1, x2, y2))

# 클래스 0, 3, 4번: 신뢰도가 가장 높은 영역만 선택
selected_detections = []
for class_id in [0, 3, 4]:
    if detections_by_class[class_id]:
        best_box = max(detections_by_class[class_id], key=lambda x: x[0])  # 신뢰도 기준 최댓값
        selected_detections.append((class_id, *best_box))

# 클래스 1번은 모든 결과를 추가 (임계값을 만족하는 모든 검출 결과)
for detection in detections_by_class[1]:
    selected_detections.append((1, *detection))

# 클래스별 색상 설정
class_colors = {
    0: (255, 0, 0),    # 빨강
    1: (0, 255, 0),    # 초록
    3: (0, 0, 255),    # 파랑
    4: (255, 255, 0)   # 노랑
}

# EasyOCR Reader 초기화 (한국어 지원)
reader = Reader(['ko', 'en'])

# 선택된 영역에서 EasyOCR 텍스트 인식 수행
for detection in selected_detections:
    class_id, confidence, x1, y1, x2, y2 = detection

    # 검출된 영역 자르기
    cropped_image = image[y1:y2, x1:x2]

    # 이미지 크기 조정 함수 적용
    cropped_image = resize_for_ocr(cropped_image)

    # EasyOCR로 텍스트 인식
    ocr_result = reader.readtext(cropped_image)

     # 텍스트를 하나의 문장으로 합치기
    combined_text = " ".join([text for _, text, _ in ocr_result])


    print(f"\n[Detection] Class: {class_id}, Confidence: {confidence}")
    for bbox, text, text_conf in ocr_result:
        print(f"Detected Text: {text}, Confidence: {text_conf}")



0: 640x480 1 date_time, 1 item, 1 receipt, 1 total, 234.0ms
Speed: 2.5ms preprocess, 234.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

[Detection] Class: 0, Confidence: 0.8465574383735657
Detected Text: 2022-06-22, Confidence: 0.8919151120107556
Detected Text: 18:56:28, Confidence: 0.8791125589447442

[Detection] Class: 4, Confidence: 0.868303656578064
Detected Text: 합 계 금액, Confidence: 0.6370755281372791
Detected Text: 5,500, Confidence: 0.9997636645409276

[Detection] Class: 1, Confidence: 0.8734450340270996
Detected Text: 수제바닐라반라떼, Confidence: 0.36531331590879956


## 4.1 YOLO_Paddle OCR Pipeline

In [2]:
import cv2
from ultralytics import YOLO
from paddleocr import PaddleOCR
import numpy as np

# YOLOv8 모델 로드
model = YOLO('best.pt')

# 테스트 이미지 경로
image_path = "test_image.jpg"
image = cv2.imread(image_path)

# YOLOv8으로 이미지 처리
results = model(image)
result = results[0]  # 단일 이미지 처리 시

# 클래스별로 검출 결과 저장
detections_by_class = {0: [], 1: [], 3: [], 4: []}
confidence_threshold = 0.45

# YOLO 모델 결과에서 바운딩 박스 추출
for box in result.boxes:
    confidence = box.conf[0].item()
    class_id = int(box.cls[0].item())

    if class_id in detections_by_class and confidence >= confidence_threshold:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        detections_by_class[class_id].append((confidence, x1, y1, x2, y2))

# 클래스 0, 3, 4: 신뢰도 가장 높은 영역만 선택
selected_detections = []
for class_id in [0, 3, 4]:
    if detections_by_class[class_id]:
        best_box = max(detections_by_class[class_id], key=lambda x: x[0])
        selected_detections.append((class_id, *best_box))

# 클래스 1: 모든 결과 추가
selected_detections.extend([(1, *det) for det in detections_by_class[1]])

# PaddleOCR Reader 초기화
ocr = PaddleOCR(use_angle_cls=True, lang='korean', det_db_box_thresh=0.5)

# 선택된 영역에서 OCR 수행
for detection in selected_detections:
    class_id, confidence, x1, y1, x2, y2 = detection

    # 검출된 영역 자르기 (BGR -> RGB 변환)
    cropped_image = cv2.cvtColor(image[y1:y2, x1:x2], cv2.COLOR_BGR2RGB)
    # PaddleOCR로 텍스트 인식
    ocr_result = ocr.ocr(cropped_image, det=False)

    # 결과 처리
    combined_text = ""
    if ocr_result:
        combined_text = " ".join(
            [text_info[0] for line in ocr_result for text_info in line]
        )
        for line in ocr_result:
            for text_info in line:
                text, conf = text_info  # 텍스트와 신뢰도 추출
                print(f"Detected Text: {text}, Class: {class_id}, Confidence: {conf}")
  


0: 640x480 1 date_time, 3 items, 1 receipt, 1 store, 1 total, 283.2ms
Speed: 10.3ms preprocess, 283.2ms inference, 11.9ms postprocess per image at shape (1, 3, 640, 480)
download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/Multilingual_PP-OCRv3_det_infer.tar to /Users/minseokim/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer/Multilingual_PP-OCRv3_det_infer.tar


100%|██████████| 3762/3762 [00:01<00:00, 3107.98it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/multilingual/korean_PP-OCRv4_rec_infer.tar to /Users/minseokim/.paddleocr/whl/rec/korean/korean_PP-OCRv4_rec_infer/korean_PP-OCRv4_rec_infer.tar


100%|██████████| 23810/23810 [00:05<00:00, 4233.78it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /Users/minseokim/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:00<00:00, 2543.58it/s]

[2024/12/12 04:49:34] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/minseokim/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.5, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/minseokim/.paddleocr/whl/rec/korean/korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', re

Detected Text: 20241-21 13:13:46 , Class: 0, Confidence: 0.9183201193809509
Detected Text: Leafful, Class: 3, Confidence: 0.9767846465110779
Detected Text: 합게12000, Class: 4, Confidence: 0.7319976687431335
Detected Text: 크림카라멜D0, Class: 1, Confidence: 0.8445944786071777
Detected Text: CafHot, Class: 1, Confidence: 0.9926131367683411


'\n# 검출 영역 시각화 (원본 이미지에 표시)\n    color = class_colors.get(class_id, (0, 255, 255))  # 기본값: 하늘색\n    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)\n\n    # 텍스트 추가 (Pillow 사용)\n    if combined_text:\n        pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # OpenCV 이미지를 PIL로 변환\n        draw = ImageDraw.Draw(pil_image)\n        font_path = "/content/PretendardVariable.ttf"  # 한글 폰트 파일 경로\n        font = ImageFont.truetype(font_path, 50)  # 폰트 크기 조정\n\n        # 텍스트 표시 (텍스트가 박스 영역 밖으로 나가지 않도록 y1 위치 조정)\n        draw.text((x1, max(y1 - 30, 0)), combined_text, font=font, fill=(255, 255, 255))  # 흰색 텍스트\n\n        # PIL 이미지를 다시 OpenCV 이미지로 변환\n        image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)\n\n# 최종 이미지 표시 (Colab 환경)\nfrom google.colab.patches import cv2_imshow\ncv2_imshow(image)  # OpenCV 이미지 표시\n'

## 5. Final **YOLO_PaddleOCR** Pipeline (+Post-Processing)

In [1]:
import cv2
from ultralytics import YOLO
from paddleocr import PaddleOCR
import numpy as np
from datetime import datetime

# YOLOv8 모델 로드
model = YOLO('best.pt')

# 테스트 이미지 경로
image_path = "test_image.jpg"
image = cv2.imread(image_path)

# YOLOv8으로 이미지 처리
results = model(image)
result = results[0]  # 단일 이미지 처리 시

# 클래스별로 검출 결과 저장
detections_by_class = {0: [], 1: [], 3: [], 4: []}
confidence_threshold = 0.45

# YOLO 모델 결과에서 바운딩 박스 추출
for box in result.boxes:
    confidence = box.conf[0].item()
    class_id = int(box.cls[0].item())

    if class_id in detections_by_class and confidence >= confidence_threshold:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        detections_by_class[class_id].append((confidence, x1, y1, x2, y2))

# 클래스 0, 3, 4: 신뢰도 가장 높은 영역만 선택
selected_detections = []
for class_id in [0, 3, 4]:
    if detections_by_class[class_id]:
        best_box = max(detections_by_class[class_id], key=lambda x: x[0])
        selected_detections.append((class_id, *best_box))

# 클래스 1: 모든 결과 추가
selected_detections.extend([(1, *det) for det in detections_by_class[1]])

# PaddleOCR Reader 초기화
ocr = PaddleOCR(use_angle_cls=True, lang='korean', det_db_box_thresh=0.5)

# 구조화된 데이터 저장
structured_data = {
    "store_name": None,
    "date_time": None,
    "total": None,
    "items": []
}

# 선택된 영역에서 OCR 수행
for detection in selected_detections:
    class_id, confidence, x1, y1, x2, y2 = detection

    # 검출된 영역 자르기 (BGR -> RGB 변환)
    cropped_image = cv2.cvtColor(image[y1:y2, x1:x2], cv2.COLOR_BGR2RGB)
    # PaddleOCR로 텍스트 인식
    ocr_result = ocr.ocr(cropped_image, det=False)

    # 결과 처리
    combined_text = ""
    if ocr_result:
        combined_text = " ".join(
            [text_info[0] for line in ocr_result for text_info in line]
        )
        if class_id == 0:  # 날짜 처리
            try:
                structured_data["date_time"] = datetime.strptime(
                    combined_text, "%Y-%m-%d %H:%M:%S"
                )
            except ValueError:
                print(f"Invalid date format: {combined_text}")
        elif class_id == 4:  # 총 금액 숫자만 추출
            total_match = "".join(filter(str.isdigit, combined_text))
            structured_data["total"] = int(total_match) if total_match.isdigit() else None
        elif class_id == 1:  # 품목
            structured_data["items"].append(combined_text)
        elif class_id == 3:  # 상호명
            structured_data["store_name"] = combined_text

# 최종 구조화된 데이터 출력
print("=== Structured Data ===")
print(f"Store Name: {structured_data['store_name']}")
print(f"Date and Time: {structured_data['date_time']}")
print(f"Total Amount: {structured_data['total']:,} 원" if structured_data["total"] else "Total Amount: Not detected")
print("Items:")
for item in structured_data["items"]:
    print(f"  - {item}")



0: 640x480 1 date_time, 1 item, 1 receipt, 1 total, 267.7ms
Speed: 5.9ms preprocess, 267.7ms inference, 8.2ms postprocess per image at shape (1, 3, 640, 480)
[2024/12/17 20:00:35] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/minseokim/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.5, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorit